In [3]:
# @title A) Config, input & preview (edit blacklist here)
pdb_ids = "" # @param {"type":"string"}

# --- Filtering rules ---
BLACKLIST = {
    # Common solvents/buffers/cryoprotectants
    "EDO","DMS","GOL","MPD","PEG","P33","P4G","TRS","MES",
    "HEPES","EPE","ACT","ACE","FMT","IPA","BU3","PLP",
    "AF3","MLI","OGA","CIT","PG4",
    # Glycans
    "NAG", "BMA","GAL",
    # Inorganic anions/cations
    "SO4","PO4","CL","BR","I","NO3","SCN","CO3","NA",
    "K","MG","CA","ZN","MN","FE","CO","CU","NI","CD","IOD",
    # Frequent artifacts / modified residues
    "MSE","SEP","TPO","PTR","UNL","UNX",
    # Water
    "HOH"
}

# Exclude ligands with no carbon atoms in their empirical formula (inorganic ions)
EXCLUDE_NO_CARBON = True  # @param {type:"boolean"}

import io, requests, pandas as pd

# ---------- helpers ----------
def fetch(url, timeout=30):
    try:
        r = requests.get(url, timeout=timeout); r.raise_for_status()
        return r.json()
    except Exception:
        return None

def pdbe(endpoint, pdb_id):
    data = fetch(f"https://www.ebi.ac.uk/pdbe/api/pdb/entry/{endpoint}/{pdb_id.lower()}")
    if isinstance(data, dict):
        return data.get(pdb_id.lower()) or data.get(pdb_id.upper()) or data
    return data

def parse_ids(s: str):
    if not s: return []
    toks = [t.strip().upper() for t in s.replace(",", " ").split() if len(t.strip()) >= 4]
    seen, out = set(), []
    for t in toks:
        if t not in seen:
            seen.add(t); out.append(t)
    return out

def get_title(pdb_id: str):
    d = pdbe("summary", pdb_id)
    return (d[0].get("title") if isinstance(d, list) and d else None) or "NA"

def get_ligand_records(pdb_id: str):
    """
    Returns list of dicts: {chem_comp_id, name, formula}
    """
    d = pdbe("ligand_monomers", pdb_id)
    recs = []
    if isinstance(d, list):
        for x in d:
            chem = (x.get("chem_comp_id") or "").strip().upper()
            if not chem: continue
            recs.append({
                "chem_comp_id": chem,
                "name": x.get("name"),
                "formula": x.get("formula")
            })
    return recs

def should_filter(rec):
    chem = rec["chem_comp_id"].upper()
    if chem in BLACKLIST:
        return True, "blacklist"
    if EXCLUDE_NO_CARBON:
        f = (rec.get("formula") or "").upper()
        if f and "C" not in f:  # no carbon atoms → inorganic ion
            return True, "no-carbon"
    return False, ""

# ---------- gather IDs (typed + optional CSV/Excel upload; any orientation) ----------
typed_ids = parse_ids(pdb_ids)
uploaded_ids = []

try:
    from google.colab import files
    print("Optional: upload CSV or Excel (.csv, .xlsx, .xls) with PDB IDs anywhere in the sheet.")
    up = files.upload()

    def read_any_table(fname, content):
        bio = io.BytesIO(content)
        if fname.lower().endswith(".csv"):
            return pd.read_csv(bio, header=None)
        elif fname.lower().endswith((".xlsx", ".xls")):
            return pd.read_excel(bio, header=None)  # first sheet
        else:
            raise ValueError("Unsupported file type")

    if up:
        flat_vals = []
        for fname, content in up.items():
            try:
                df_in = read_any_table(fname, content)
                flat_vals.extend(df_in.astype(str).values.ravel().tolist())
            except Exception as e:
                print(f"Could not read {fname}: {e}")
        uploaded_ids = parse_ids(" ".join(flat_vals))
        if uploaded_ids:
            print(f"Found {len(uploaded_ids)} PDB IDs from upload.")
except Exception:
    pass

# merge + de-duplicate
pdb_list = parse_ids(" ".join(typed_ids + uploaded_ids))
if not pdb_list:
    raise ValueError("Provide at least one valid PDB ID (e.g., 5GY2).")

# ---------- preview ligands & filtering outcome ----------
preview_rows = []
for pid in pdb_list:
    title = get_title(pid)
    recs = get_ligand_records(pid)
    # collapse duplicates by chem_comp_id, keep first name/formula
    seen = {}
    for r in recs:
        key = r["chem_comp_id"]
        if key not in seen:
            seen[key] = r
    recs = list(seen.values())
    for r in recs:
        filt, reason = should_filter(r)
        preview_rows.append({
            "pdb_id": pid,
            "Title": title,
            "chem_comp_id": r["chem_comp_id"],
            "name": r.get("name"),
            "formula": r.get("formula"),
            "filtered": filt,
            "reason": reason
        })

preview_df = pd.DataFrame(preview_rows).sort_values(["pdb_id","filtered","chem_comp_id"])
print(f"PDBs: {len(pdb_list)} | unique ligands listed: {preview_df.shape[0]}")
display(preview_df.head(30))  # quick peek

# Summary counts per PDB
summary = (preview_df
           .groupby("pdb_id")["filtered"]
           .value_counts()
           .unstack(fill_value=0)
           .rename(columns={False:"kept", True:"filtered"})
           .reset_index())
print("Per-PDB ligand counts (kept vs filtered):")
display(summary)

Optional: upload CSV or Excel (.csv, .xlsx, .xls) with PDB IDs anywhere in the sheet.


Saving PDB _TEST_targets.xlsx to PDB _TEST_targets (1).xlsx
Found 12 PDB IDs from upload.
PDBs: 12 | unique ligands listed: 33


,pdb_id,Title,chem_comp_id,name,formula,filtered,reason
9,4C02,Crystal structure of human ACVR1 (ALK2) in com...,FLC,None,None,False,
10,4C02,Crystal structure of human ACVR1 (ALK2) in com...,TAK,None,None,False,
11,4C02,Crystal structure of human ACVR1 (ALK2) in com...,EDO,None,None,True,blacklist
1,5L78,Crystal structure of human aminoadipate semial...,NAD,None,None,False,
0,5L78,Crystal structure of human aminoadipate semial...,EDO,None,None,True,blacklist
2,5O1N,Crystal structure of human aminoadipate semial...,9H8,None,None,False,
3,5O1N,Crystal structure of human aminoadipate semial...,DMS,None,None,True,blacklist
4,5O1N,Crystal structure of human aminoadipate semial...,EDO,None,None,True,blacklist
5,5O1N,Crystal structure of human aminoadipate semial...,PEG,None,None,True,blacklist
7,5O1O,Crystal structure of human aminoadipate semial...,PRO,None,None,False,


Per-PDB ligand counts (kept vs filtered):


filtered,pdb_id,kept,filtered
0,4C02,2,1
1,5L78,1,1
2,5O1N,1,3
3,5O1O,1,1
4,5OXG,1,2
5,6GI6,1,2
6,6GIN,1,2
7,6GIP,1,2
8,6ZGC,1,2
9,7NNS,1,2


In [4]:
# @title B) Build filtered Excel sheet & download
excel_filename = "TEST_pdb_ligands_smiles_filtered.xlsx"  # @param {type:"string"}

import requests, pandas as pd

# depend on objects from Cell A: pdb_list, BLACKLIST, EXCLUDE_NO_CARBON, get_title, get_ligand_records, should_filter

def fetch(url, timeout=30):
    try:
        r = requests.get(url, timeout=timeout); r.raise_for_status()
        return r.json()
    except Exception:
        return None

_smiles_cache = {}
def get_smiles(chem_id: str):
    key = chem_id.upper()
    if key in _smiles_cache: return _smiles_cache[key]
    # RCSB first
    s = None
    rcsb = fetch(f"https://data.rcsb.org/rest/v1/core/chemcomp/{key}")
    if isinstance(rcsb, dict):
        desc = rcsb.get("rcsb_chem_comp_descriptor") or {}
        s = desc.get("smiles") or desc.get("smiles_stereo")
    # PDBe fallback
    if not s:
        pdbe_cmp = fetch(f"https://www.ebi.ac.uk/pdbe/graph-api/compound/summary/{key}")
        if isinstance(pdbe_cmp, dict):
            blk = pdbe_cmp.get(key) or pdbe_cmp.get(key.lower())
            if isinstance(blk, list) and blk:
                s = blk[0].get("smiles")
    _smiles_cache[key] = s or pd.NA
    return _smiles_cache[key]

rows = []
for pid in pdb_list:
    title = get_title(pid)
    recs = get_ligand_records(pid)
    # collapse duplicates by chem_comp_id
    seen = {}
    for r in recs:
        key = r["chem_comp_id"]
        if key not in seen:
            seen[key] = r
    recs = list(seen.values())
    # apply filters
    kept = [r for r in recs if not should_filter(r)[0]]

    if kept:
        for r in kept:
            chem = r["chem_comp_id"]
            rows.append({
                "pdb_id": pid,
                "Title": title,
                "chem_comp_id": chem,
                "smiles": get_smiles(chem)
            })
    else:
        # if nothing kept after filtering, still emit a row to mark the PDB
        rows.append({"pdb_id": pid, "Title": title, "chem_comp_id": pd.NA, "smiles": pd.NA})

df = pd.DataFrame(rows, columns=["pdb_id","Title","chem_comp_id","smiles"])

with pd.ExcelWriter(excel_filename, engine="openpyxl") as w:
    df.to_excel(w, index=False, sheet_name="PDB")

print(f"Created: {excel_filename} | Rows: {len(df)}")

# auto-download (Colab)
try:
    from google.colab import files
    files.download(excel_filename)
except Exception:
    pass


Created: TEST_pdb_ligands_smiles_filtered.xlsx | Rows: 13


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>